In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

/opt/anaconda3/envs/ai574-pocs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0622 20:52:43.731000 9501 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


# Load model
https://huggingface.co/luqh/ClinicalT5-large

In [21]:
MODEL_ID = "luqh/ClinicalT5-large"
tokenizer = T5Tokenizer.from_pretrained(MODEL_ID, model_max_length = 1024)
model = T5ForConditionalGeneration.from_pretrained(MODEL_ID, from_flax = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All Flax model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the Flax model and are newly initialized: ['lm_head.weight', 'decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Some medical text

Not that this will fail if there are too many tokens. We'll have to do some chunking or RAG in order to
deal with the small context window.

In [22]:
medical_text = """
Non-steroidal anti-inflammatory drugs are not only potent analgesics and antipyretics but also nephrotoxins, and may cause 
electrolyte disarray. In addition to the commonly expected effects, including hyperkalemia, hyponatremia, acute renal injury, 
renal cortical necrosis, and volume retention, glomerular disease with or without nephrotic syndrome or nephritis can occur as 
well including after years of seemingly safe administration. Minimal change disease, secondary membranous glomerulonephritis, 
and acute interstitial nephritis are all reported glomerular lesions seen with non-steroidal anti-inflammatory use. We report a 
patient who used non-steroidal anti-inflammatory drugs for years without diabetes, chronic kidney disease, or proteinuria; he 
then developed severe nephrotic range proteinuria with 7 g of daily urinary protein excretion. Renal biopsy showed minimal 
change nephropathy, a likely secondary membranous glomerulonephritis, and acute interstitial nephritis present simultaneously
in one biopsy. 

"""

In [23]:
len(medical_text)

1030

# Inference

In [34]:
prompt = f"summarize: {medical_text.strip()}"
inputs = tokenizer(prompt, return_tensors = "pt", truncation = False)

with torch.no_grad():
    summary_ids = model.generate(**inputs,
        max_new_tokens = 128, num_beams = 4, length_penalty = 1.5, early_stopping = False, no_repeat_ngram_size = 3, encoder_no_repeat_ngram_size = 3)

In [35]:
for summary_id in summary_ids:
    summary = tokenizer.decode(summary_id, skip_special_tokens = True)
    print(summary)

and acute renal failure can also occur with long-term use. also potent potent potent narcotics are they can be a cause of renal failure.,,,. acute with are acute glomer,al change disease with,, secondary and developed severe with minimal change disease and a, and minimal change and acute. renal biopsy showed all three lesions present in the same biopsy. this
